# MobileNet in Keras

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!/opt/bin/nvidia-smi

#### Learn more about the paper : https://arxiv.org/pdf/1704.04861.pdf

In [ ]:
class Conv(tf.keras.layers.Layer):
    def __init__(self, input_shape, ks, s, filters):
        super(Conv, self).__init__()
        self.conv = tf.keras.layers.Conv2D(input_shape=input_shape,
                                   kernel_size = ks,
                                   strides = s,
                                   padding = 'same',
                                   use_bias = False,
                                   filters = filters)
        self.bn = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.Activation('relu')
        
    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        return self.relu(x)

In [ ]:
class ConvDW(tf.keras.layers.Layer):
    def __init__(self, ks, s, filters):
        super(ConvDW, self).__init__()
        self.dw = tf.keras.layers.DepthwiseConv2D(
            kernel_size = ks,
            strides = (s, s),
            padding = 'same',
            use_bias = False,
        )
        self.b1 = tf.keras.layers.BatchNormalization()
        self.r1 = tf.keras.layers.Activation('relu')
        self.pw = tf.keras.layers.Conv2D(
            kernel_size=(1, 1),
            strides = (1,1),
            padding='same',
            use_bias = False,
            filters = filters)
        self.b2 = tf.keras.layers.BatchNormalization()
        self.r2 = tf.keras.layers.Activation('relu')
        
    def call(self, inputs):
        x = self.dw(inputs)
        x = self.b1(x)
        x = self.r1(x)
        
        x = self.pw(x)
        x = self.b2(x)
        return self.r2(x)

In [ ]:
class MobileNet(tf.keras.Model):
    def __init__(self, alpha=1, rho=1, n_classes=1000, in_channels=3):
        super(MobileNet, self).__init__()
        
        model = tf.keras.Sequential()
        model.add(Conv((224, 224, 3), (3, 3), (2,2), 32))
        
        layer = [
                (32, 64, (3, 3), 1),
                (64, 128, (3, 3), 2),
                (128, 128, (3, 3), 1),
                (128, 128, (3, 3), 2),
                (128, 256, (3, 3), 1),
                (256, 512, (3, 3), 2),
                *[(512, 512, (3, 3), 1) for _ in range(5)],
                (512, 1024, (3,3), 2),
                (1024, 1024, (3, 3), 2),
                ]
        for i in range(len(layer)):
            in_channels, out_channels, kernel_size, stride = layer[i]
            model.add(ConvDW(ks = kernel_size, s = stride, filters=out_channels))
        
        model.add(tf.keras.layers.GlobalAveragePooling2D())
        model.add(tf.keras.layers.Dense(units = n_classes,
                                        activation='softmax'))
        self.model = model

    def train(self, X_train, y_train, X_val,
              training_generator, validation_generator, batch_size=32, epochs = 15):
        self.model.compile(optimizer="rmsprop",
                          loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                           metrics=['accuracy'])
        self.model.fit_generator(generator=training_generator,
                                 steps_per_epoch = int(len(X_train) // batch_size),
                                 validation_data = validation_generator,
                                 verbose = 1,
                                 validation_steps = int(len(X_val) // batch_size))
    def predict(self, X):
        return self.model.predict(X)

In [ ]:
# NGL, ImageNet quite heavy, let's try Stanford Dogs
# http://vision.stanford.edu/aditya86/ImageNetDogs/

# Training part, all credits goes to 
# https://medium.com/@mrgarg.rajat/training-on-large-datasets-that-dont-fit-in-memory-in-keras-60a974785d71

In [ ]:
!tar -xvf '/content/images.tar'

In [ ]:
import os
import shutil
import cv2

train_dir = '/content/Images/'
dest_dir = '/content/all_images/'

filenames = []
labels = []
labels_counter = 0


os.makedirs((os.path.dirname(dest_dir)), exist_ok=True)
for subdir, dirs, files in os.walk(train_dir):
  for file in files:
    full_path = os.path.join(subdir, file)
    shutil.copy(full_path, dest_dir + file)
    filenames.append(file)
    labels.append(labels_counter)
  labels_counter += 1

from sklearn.utils import shuffle

filenames_shuffled, y_labels_shuffled = shuffle(filenames, labels)

from sklearn.model_selection import train_test_split

filenames_shuffled = np.array(filenames_shuffled)
y_labels_shuffled = np.array(y_labels_shuffled)

X_train_filenames, X_val_filenames, y_train, y_val = train_test_split(filenames_shuffled, y_labels_shuffled,
                                                                      test_size=0.2, random_state=42)

In [ ]:
class My_Custom_Generator(tf.keras.utils.Sequence):
  def __init__(self, image_filenames, labels, batch_size):
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size

  def __len__(self):
    return(np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)

  def __getitem__(self, idx):
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    X = []
    for file_name in batch_x:
      X.append(
          (cv2.resize(
              cv2.imread(dest_dir + file_name),
              (224, 224)
          )) / 255.0 
      )
    return np.array(X), np.array(batch_y)

In [ ]:
my_training_batch_generator = My_Custom_Generator(X_train_filenames, y_train, batch_size=32)
my_validation_batch_generator = My_Custom_Generator(X_val_filenames, y_val, batch_size=32)

In [ ]:
print(X_train_filenames)
print(y_train)
print(labels_counter)

In [ ]:
model = MobileNet(n_classes=labels_counter)

In [ ]:
model.train(X_train_filenames, y_train, X_val_filenames, my_training_batch_generator,
            my_validation_batch_generator, batch_size=32, epochs=15)